テストデータTD用

In [1]:
from asyncio.windows_events import NULL
import pandas as pd
import numpy as np
import re
import scipy
from scipy.spatial.transform import Rotation
from scipy.ndimage import rotate
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import japanize_matplotlib  # 日本語フォントを有効にするためのimport
import os
import csv
import cv2
from matplotlib.patches import Ellipse
import math
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import LeaveOneOut
import glob
from torch.utils.data import TensorDataset, DataLoader

# モデルの定義
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
class MyLSTM(nn.Module):
    def __init__(self, feature_size, hidden_dim, n_layers, output_size,dropout_prob=0.2):
        super(MyLSTM, self).__init__()

        self.feature_size = feature_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.output_size = output_size
        self.predict_frame = 10  # 予測するフレーム数

        self.lstm = nn.LSTM(feature_size, hidden_dim, n_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_size * self.predict_frame)

        # Initialize h_0 and c_0 as class attributes
        self.h_0 = nn.Parameter(torch.zeros(n_layers * 2, 1, hidden_dim))
        self.c_0 = nn.Parameter(torch.zeros(n_layers * 2, 1, hidden_dim))

        self.dropout = nn.Dropout(dropout_prob)


        # Move LSTM and Linear layers to device
        self.to(device)

    def forward(self, x):
        h_0 = torch.zeros(self.n_layers * 2, x.size(0), self.hidden_dim).to(x.device)
        c_0 = torch.zeros(self.n_layers * 2, x.size(0), self.hidden_dim).to(x.device)
        output, (hn, cn) = self.lstm(x, (h_0, c_0))

        hn = hn.view(self.n_layers, 2, x.size(0), self.hidden_dim)[-1].view(x.size(0), -1)
        y = self.fc(hn)
        y = y.view(-1, self.predict_frame, self.output_size)

        return y


In [3]:
import torch
import numpy as np

def apply_median_filter_truncate(tensor):
    # テンソルの形状を取得
    rows, cols = tensor.shape

    # 出力用のテンソルを初期化（最初と最後の行を除外）
    filtered_tensor = torch.zeros((rows - 2, cols), dtype=torch.float32)

    # メディアンフィルタを適用（最初と最後の行は除外）
    for i in range(1, rows - 1):
        for j in range(cols):
            # 現在の要素とその前後の要素を取得
            values = tensor[i-1:i+2, j]

            # 中央値を計算
            median_val = torch.median(values)
            filtered_tensor[i-1, j] = median_val

    return filtered_tensor




In [4]:
def train_model(train_loader, epochs = 80):

    # モデルのパラメータ
    feature_size = 7
    hidden_dim = 64
    n_layers = 1
    output_size = 1  # 予測する次元数

    # モデルのインスタンス化
    model = MyLSTM(feature_size, hidden_dim, n_layers, output_size)
    criterion = nn.MSELoss()  # 損失関数
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # DataLoaderの設定
    batch_size = 1  # 適切なバッチサイズを設定

    model.to(device)


    for epoch in range(epochs):
        model.train()
        epoch_loss = 0

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            y_pred = model(x)
            loss = criterion(y_pred, y)
            epoch_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss/len(train_loader)}')

    return model

In [11]:
from sklearn.preprocessing import MinMaxScaler
import os
import glob
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
import csv

parts = ['R_hip', 'L_hip', 'R_ankle', 'L_ankle', 'shoulder', 'R_elbow', 'L_elbow', 'R_midhip', 'L_midhip', 'neck', 'R_knee', 'L_knee']

def process_body_part(body_part):
    scaler = MinMaxScaler()

    all_subdirectories = [os.path.abspath(path) for path in glob.glob("開始2秒/TD/nagasaki_*/*")]
    
    all_file_paths = [os.path.abspath(path) for path in glob.glob("開始2秒/TD/nagasaki_*/*/*/*.csv")]
    output_base_path = f'開始2秒/{body_part}_model'

    for exclude_dir in all_subdirectories:
        print(exclude_dir)
        exclude_paths = [os.path.abspath(path) for path in glob.glob(exclude_dir + '/*/*.csv')]
        
        # exclude_paths内のファイルをfile_pathsから除外する
        file_paths = [path for path in all_file_paths if not any(path.startswith(exclude_path) for exclude_path in exclude_paths)]

        print(exclude_paths)


        
        dataframes = []
        for path in file_paths:
            data = pd.read_csv(path)
            data = data.iloc[:, 1:]
            dataframes.append(data)

        # Scaling
        for df in dataframes:
            df.iloc[:, 0:] = scaler.fit_transform(df.iloc[:, 0:])

        tensor_list = [torch.tensor(df.values, dtype=torch.float32) for df in dataframes]
        filtered_tensor_list = [apply_median_filter_truncate(tensor) for tensor in tensor_list]

        window_size = 28
        predict_frame = 10

        train_input_tensor_list = []
        train_target_tensor_list = []

        for item in filtered_tensor_list:
            if len(item) >= window_size + predict_frame:
                for i in range(len(item) - window_size - predict_frame + 1):
                    train_input_tensor_list.append(item[i:i + window_size, 0:])

                    # 部位に応じてスライシングを行う
                    part_index = parts.index(body_part)
                    train_target_tensor_list.append(item[i + window_size:i + window_size + predict_frame, part_index:part_index + 1])

        train_input_tensor = torch.stack(train_input_tensor_list)
        train_target_tensor = torch.stack(train_target_tensor_list)

        train_dataset = TensorDataset(train_input_tensor, train_target_tensor)

        batch_size = 1
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        trained_model = train_model(train_loader)

        parts_of_path = exclude_dir.split(os.sep)
        model_name = f"NEW_Except_{parts_of_path[-2]}_{parts_of_path[-1]}"

        model_save_path = os.path.join(output_base_path, f"{model_name}.pth")
        torch.save(trained_model.state_dict(), model_save_path)


In [12]:
for part in parts:
    process_body_part(part)

c:\Users\yohmo\additional\開始2秒\TD\nagasaki_2022_10\AM_10
['c:\\Users\\yohmo\\additional\\開始2秒\\TD\\nagasaki_2022_10\\AM_10\\右足1\\開始2秒_右足1_角度.csv', 'c:\\Users\\yohmo\\additional\\開始2秒\\TD\\nagasaki_2022_10\\AM_10\\右足2\\開始2秒_右足2_角度.csv', 'c:\\Users\\yohmo\\additional\\開始2秒\\TD\\nagasaki_2022_10\\AM_10\\左足1\\開始2秒_左足1_角度.csv', 'c:\\Users\\yohmo\\additional\\開始2秒\\TD\\nagasaki_2022_10\\AM_10\\左足2\\開始2秒_左足2_角度.csv']
Epoch 1/80, Loss: 0.10739022029730945
Epoch 2/80, Loss: 0.07764645254908081
Epoch 3/80, Loss: 0.07583188343112572
Epoch 4/80, Loss: 0.0647643534006933
Epoch 5/80, Loss: 0.05773232632931561
Epoch 6/80, Loss: 0.054989602669285466
Epoch 7/80, Loss: 0.053056505406335094
Epoch 8/80, Loss: 0.05118088799361617
Epoch 9/80, Loss: 0.047617629252804015
Epoch 10/80, Loss: 0.04810312644351656
Epoch 11/80, Loss: 0.047310029668733475
Epoch 12/80, Loss: 0.04491444161961763
Epoch 13/80, Loss: 0.04418161416408391
Epoch 14/80, Loss: 0.04371884133672724
Epoch 15/80, Loss: 0.0432819231114804
Epoch 16